In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import ky_utils.ky_interpretable_functions as interpret
import ky_utils.ky_riskslim as slim
import ky_utils.ky_stumps as stumps

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model
from sklearn.metrics import roc_auc_score

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\ky-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['felony_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['felony_two_year'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.3]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [4]:
np.mean(cart_summary['FL_score']), np.mean(ebm_summary['FL_score'])

(0.6078481149790553, 0.62039397659974)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-stumps.csv").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-stumps.csv").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_two_year'].values

## columns 
cols = KY_X.columns[1:]

In [8]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.002, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge19',
 'age_at_current_charge20',
 'age_at_current_charge27',
 'age_at_current_charge30',
 'age_at_current_charge36',
 'age_at_current_charge40',
 'age_at_current_charge43',
 'age_at_current_charge47',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_arrest6',
 'p_charges1',
 'p_violence1',
 'p_felony1',
 'p_misdemeanor1',
 'p_property1',
 'p_drug2',
 'p_drug3',
 'p_stealing1',
 'p_fta_two_year1',
 'p_fta_two_year2',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'one_year1',
 'current_violence1']

In [9]:
c_grid={'C': [0.001, 0.002]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [10]:
stumps_summary['best_params'], np.mean(stumps_summary['FL_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}],
 0.6220579950888343,
 0.0004761773995796803)

### RiskSLIM

In [11]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.0005, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

13

In [12]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [13]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'felony_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset = 100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
02/15/20 @ 07:38 PM | 183 rows in lookup table
02/15/20 @ 07:38 PM | ------------------------------------------------------------
02/15/20 @ 07:38 PM | runnning initialization procedure
02/15/20 @ 07:38 PM | ------------------------------------------------------------
02/15/20 @ 07:38 PM | CPA produced 2 cuts
02/15/20 @ 07:38 PM | running naive rounding on 5 solutions
02/15/20 @ 07:38 PM | best objective value: 0.5950
02/15/20 @ 07:38 PM | rounding produced 5 integer solutions
02/15/20 @ 07:38 PM | best objective value is 0.5475
02/15/20 @ 07:38 PM | running sequential rounding on 5 solutions
02/15/20 @ 07:38 PM | best objective value: 0.5950
02/15/20 @ 07:38 PM | sequential rounding produced 1 integer solutions
02/15/20 @ 07:38 PM | best objective value: 0.5475
02/15/20 @ 07:38 PM | polishing 6 solutions
02/15/20 @ 07:38 PM | best objective value: 0.5475
02/15/20 @ 07:38 PM | polishing produced 5 integer solutions
02/15/20 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 07:38 PM | adding 248 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3066.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Elapsed time = 36.13 sec. (39921.96 ticks, tree = 2.77 MB, solutions = 11)
 173189  8084        0.2924     1        0.2933        0.2913   584306    0.69%        alpha_12 U 173189 151424     40
 177722  7107        0.2916     3        0.2933        0.2916   595019    0.61%           rho_7 D 177722 177721     34
 181906  5600        cutoff              0.2933        0.2919   605088    0.49%           rho_3 U 181906  65915     24
 186262  3992        0.2923     5        0.2933        0.2923   614225    0.36%          rho_11 U 186262 122338     30
 190464  2325        cutoff              0.2933        0.2926   622860    0.25%           rho_7 U 190464  67119     26
 194822   317        cutoff              0.2933        0.2932   630982    0.04%           rho_7 U 194822 194821     19

Gomory fractional cuts applied:  1
User cuts applied:  481

Root node processing (before b&c):
  Real time             =    0.13 sec. (1.36 ticks)
Sequential b&c:
  Real time             =   40.28 sec. (45779.9

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 07:39 PM | adding 242 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3055.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 179750 29501        0.2866     8        0.2923        0.2866   730992    1.95%         alpha_8 U 179750 126947     35
 183720 29586        0.2881     1        0.2923        0.2867   742785    1.91%          rho_12 D 183720 183718     32
 187340 29344        0.2894     4        0.2923        0.2868   757137    1.88%           rho_5 U 187340 187339     31
 191546 29776        cutoff              0.2923        0.2868   769821    1.87%           rho_8 D 191546  84946     25
 195317 29598        cutoff              0.2923        0.2870   782091    1.83%           rho_8 N 195317 195316     28
 199031 29118        0.2893     7        0.2923        0.2871   795139    1.78%           rho_4 U 199031  18357     27
 202878 28337        cutoff              0.2923        0.2872   807392    1.74%         alpha_6 D 202878 202877     54
 206720 28351        0.2922     1        0.2923        0.2874   819100    1.69%           rho_7 U 206720 206719     22
 210199 27632        0.2908     1        0.2923 

...

02/15/20 @ 08:02 PM | best objective value: 0.6038
02/15/20 @ 08:02 PM | rounding produced 5 integer solutions
02/15/20 @ 08:02 PM | best objective value is 0.5497
02/15/20 @ 08:02 PM | running sequential rounding on 5 solutions
02/15/20 @ 08:02 PM | best objective value: 0.6038
02/15/20 @ 08:02 PM | sequential rounding produced 1 integer solutions
02/15/20 @ 08:02 PM | best objective value: 0.3939
02/15/20 @ 08:02 PM | polishing 6 solutions
02/15/20 @ 08:02 PM | best objective value: 0.3939
02/15/20 @ 08:02 PM | polishing produced 4 integer solutions
02/15/20 @ 08:02 PM | initialization produced 9 feasible solutions
02/15/20 @ 08:02 PM | best objective value: 0.3085
02/15/20 @ 08:02 PM | ------------------------------------------------------------
02/15/20 @ 08:02 PM | completed initialization procedure
02/15/20 @ 08:02 PM | ------------------------------------------------------------
02/15/20 @ 08:02 PM | 183 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPX

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 08:02 PM | adding 233 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3085.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.28412336390394533
CPXPARAM_MIP_Tolerances_UpperCutoff              0.3080223802065119


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 08:02 PM | adding 248 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3080.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 181174 28048        cutoff              0.2954        0.2902   718790    1.73%           rho_9 D 181174  37325     35
 185247 27745        cutoff              0.2954        0.2903   729991    1.71%         alpha_6 U 185247 185245     33
 189076 27280        cutoff              0.2954        0.2905   741588    1.66%         alpha_6 U 189076 189075     29
 192770 26770        0.2916    11        0.2954        0.2906   753275    1.62%           rho_0 D 192770 112626     26
 196270 26261        0.2907    11        0.2954        0.2907   765209    1.58%           rho_7 D 196270 196268     37
 199850 25833        0.2946     9        0.2954        0.2909   777797    1.52%           rho_7 U 199850 152660     24
 203430 25285        0.2910     7        0.2954        0.2910   789419    1.47%           rho_0 U 203430 203379     27
 206934 24268        cutoff              0.2954        0.2911   800629    1.44%           rho_7 N 206934 206932     28
 210717 23833        0.2913    12        0.2954 

In [14]:
np.mean(riskslim_summary['KY_validation']), np.mean(riskslim_summary['FL_score'])

(0.7229386144140488, 0.5854237685974288)

In [16]:
#### save results
summary_felony2_ky_inter_model = {"cart": cart_summary,
                                  "ebm": ebm_summary,
                                  "stumps": stumps_summary,
                                  "riskslim": riskslim_summary}

In [17]:
path = "./results/interpretable/two-year/"
results = [["Felony",             
            np.str(round(np.mean(cart_summary['FL_score']),3)) + " (" + np.str(round(np.std(cart_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['FL_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['FL_score']),3)) + " (" + np.str(round(np.std(stumps_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['FL_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['FL_score']),3)) + ")"]]
with open(path + 'two-year-ky-interpretable.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)